goal of this notebook is to test the capabilities of the scipy.optimize module at a very, very basic level with some toy examples.

documentation is available at: https://docs.scipy.org/doc/scipy/tutorial/optimize.html

In [1]:
import scipy.optimize as op
import numpy as np

# 0.0: Optimize x^2

In [12]:
def x_squared(x):
    return x ** 2

In [13]:
res = op.minimize(x_squared, [1234], method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 41
         Function evaluations: 82
[-9.09494702e-13]


comments: successfully finds f(0) = 0 as the minimum. this algorithm is, notably, not constrained.

# 0.1: Optimize x^2 + 5

In [18]:
def x_squared_add_five(x):
    return x ** 2 + 5

In [19]:
res = op.minimize(x_squared_add_five, [1234], method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 5.000000
         Iterations: 41
         Function evaluations: 83
[-9.09494702e-13]


comments: again, finds f(0) = 5 as the minimum.

# 0.2: Optimize (x-7)^2

In [20]:
def x_minus_seven_squared(x):
    return (x - 7) ** 2

In [21]:
res = op.minimize(x_minus_seven_squared, [1234], method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 41
         Function evaluations: 82
[7.]


comments: finds f(7) = 0 as the minimum.

# 0.3 Optimize (x-7) ^ 2, subject to x^2 < x

i am trying to use SLSQP here, which supports constraints that are relative to zero. that means, that equality constraints are of the from c(x) = 0, and ineq constraints are of the form c(x) > 0.

converting x^2 < x to this format yields: - x^2 + x > 0, whose first partial derivative is (-2x + 1)

the method needs the Jacobian of all the constraints, and it is to be provided as a dictionary (see example in the documentation: https://docs.scipy.org/doc/scipy/tutorial/optimize.html#sequential-least-squares-programming-slsqp-algorithm-method-slsqp)

In [35]:
ineq_cons = {'type': 'ineq',
             'fun' : lambda x: np.array([-x[0]**2 + x[0]]),
             'jac' : lambda x: np.array([-2.0 * x[0] + 1.0])
}

In [36]:
def x_minus_seven_squared_der(x):
    return 2 * (x - 7)


bounds = [(-1e5, 1e5)] # liberal bounds, not worried about this right now
res = op.minimize(x_minus_seven_squared, [1000], method='SLSQP', jac=x_minus_seven_squared_der,
               constraints=[ineq_cons], options={'ftol': 1e-9, 'disp': True},
               bounds=bounds)
print(res.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 35.999999999981064
            Iterations: 10
            Function evaluations: 12
            Gradient evaluations: 10
[1.]


comments: we correctly find the closest we can get to the minima (7) without messing up the constraint of (x ^ 2 < x)